## Mikroserwis

In [3]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

sys.path.append(str(Path("..").resolve()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from statsmodels.stats.power import TTestIndPower
import numpy as np
from scipy.stats import norm
import pandas as pd

TODO

## Test A/B

Oznaczenia:
- $q_L$ - jakość modelu bazowego regresji liniowej.
- $q_F$ - jakośc modelu bardziej zaawansowanego lasu losowego.

### Hipotezy

- Hipoteza zerowa $H_0$ : $q_F$ $\leq$ $q_L$ - wyniki lasu losowego nie są lepsze niż regresji liniowej.
- Hipoteza alternatywna $H_1$ : $q_F$ $\gt$ $q_L$ - wyniki lasu losowego są lepsze niż regresji liniowej.

### Przyjęte parametry

- poziom istotności:  **$\alpha$ = 0,05**
- moc testu:           **1 - $\beta$ = 0,8** <br>
Oraz na podstawie notatnika *models_03.ipynb*:
- rozmiar efektu **MDE = 5** - (poprawa jakości modelu o 5 jednostki MAE jest wystarczająca, aby uznać hipotezę zerową)
- odchylenie standardowe predykcji: **std_forest = 25** i **std_linear = 50** 

### Wyznaczanie liczby potrzebnych próbek

Do wyznaczenia liczby potrzebnych próbek posłuży test **t-Studenta**.

In [19]:
alpha = 0.05
power = 0.8
mde = 5.0
std_l = 50.0
std_f = 25.0
std_pooled = np.sqrt((std_l**2 + std_f**2) / 2)

effect_size = mde / std_pooled
analysis = TTestIndPower()
n_samples = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=1.0)

print(f"About {int(np.ceil(n_samples))} observations per model variant.")
print(f"Total sample size: {int(np.ceil(2 * n_samples))} observations.")

About 983 observations per model variant.
Total sample size: 1965 observations.


Tak duża liczba wynika z faktu, iż:
- skuteczność modeli nie różni się od siebie znacząco pod względem błędu MAE.
- dla modelu liniowego występują duże odchylenia standardowe predykcji, co wprowadza duży szum.

Potrzeba uśrednienia wielu rzeczywistych obserwacji modeli, aby móc wyciągnąć z analizy wymierne wnioski.  Dopiero taka wielkość próby gwarantuje moc testu na poziomie 0.8, pozwalając na wiarygodne odrzucenie hipotezy zerowej $H_0$ i potwierdzenie przewagi lasu losowego ($q_F$) nad modelem bazowym ($q_L$).

### Wyznaczanie wartości krytycznej

Do wyznaczenia wartości krytycznej posłużył wzór: $$\Delta_{crit} = z_{1-\alpha} \cdot SE$$
gdzie:
- **SE** - Błąd standardowy różnicy średnich odchyleń
- **$z_{1-\alpha}$** - Wartość krytyczna z rozkładu normalnego

In [20]:
se = np.sqrt((std_l**2 / n_samples) + (std_f**2 / n_samples))
z_crit = norm.ppf(1 - alpha)
delta_crit = z_crit * se

print(f"Standard error (SE): {se:.3f}")
print(f"Statistical multiplier (z): {z_crit:.3f}")
print(f"Critical value (Delta_crit): {delta_crit:.2f} MAE")

Standard error (SE): 1.784
Statistical multiplier (z): 1.645
Critical value (Delta_crit): 2.93 MAE


### Metodyka testu

W każdym zapytaniu do mikroserwisu zaszyty jest niewidoczny dla użytkownika wybór modelu, który przeprowadzi dla niego predykcję. Każdy model ma 50% szans na zostanie wybranym:
```python
random_number = random.uniform(0, 1)
if random_number < 0.5:
    model = BASE_MODEL     # Linear regression
else:
    model = ADVANCED_MODEL # Random forest
```

Przy tak dużej potrzebnej liczbie próbek zapewni to w przybliżeniu równomierne używanie modeli.

### Zbieranie logów

Mikroserwis zapisuje logi w strukturze:

*level,timestamp,model,prediction,real*

Są w nich zawarte kluczowe informacje: identyfikacja modelu, cena przewidziana przez model oraz cena rzeczywista.


### Ewaluacja testu

In [23]:
logs = pd.read_csv("../logs/service.log")

logs['abs_error'] = np.abs(logs['prediction'] - logs['real'])

group_L = logs[logs['model'] == 'linear']['abs_error']
group_F = logs[logs['model'] == 'random_forest']['abs_error']

print(f"Number of samples: Linear: {len(group_L)}, Random Forest: {len(group_F)}")
print(f"MAE: Linear: {group_L.mean():.2f}, Random Forest: {group_F.mean():.2f}")
print(f"Std Dev: Linear: {group_L.std():.2f}, Random Forest: {group_F.std():.2f}")

Number of samples: Linear: 0, Random Forest: 0
MAE: Linear: nan, Random Forest: nan
Std Dev: Linear: nan, Random Forest: nan


###